In [2]:
import pandas as pd
import os
from utils import *

In [ ]:
# Main script
if __name__ == "__main__":
    #edge_options = Options()
    driver = init_webdriver()

    try:
        # Get team name from user
        team_input = input("Enter the team for which you want to retrieve the match reports: ")
        team = normalize_team_name(team_input)
        print(f"Normalized team name: {team}")
        # Define the folder path and ensure the folder exists
        folder_path = os.path.join(os.getcwd(), "Fixtures")
        os.makedirs(folder_path, exist_ok=True)  # Create the folder if it doesn't exist
        filename = os.path.join(folder_path, f"{team}_matches_2024.xlsx")

        # Open Excel file and insert data in dataframe
        df = pd.read_excel(filename)
        # Load the matchlogs_for page and extract URLs
        url_df = load_team_urls()
        # Search for the team in the 'urls' column of the DataFrame
        matching_url = None
        for url in url_df["urls"]:
            if team in url:  # Check if the normalized team name is part of the URL
                matching_url = url
                break

        # Check if we found a matching URL, if we do execute the scraping and save team fixture data in an Excel file
        if matching_url:
            print(f"Found matching URL for {team}: {matching_url}")
            html = get_page_content(driver, matching_url)
            match_report_urls = extract_match_report_urls(html)

            # Update the main Excel file with URLs
            update_match_report_urls(df, match_report_urls,team)

            # Re-load the updated main Excel file with URLs
            df = pd.read_excel(filename)

            # Scrape and save each match report
            scrape_and_save_reports(df,driver,team)
        else:
            print(f"No matching team found for {team}.")
    finally:
        driver.quit()

In [ ]:
# Reload utils after changes
# Useful in development

import importlib
import utils
importlib.reload(utils)